In [1]:
from obspy.core.event import read_events
import obspy
import numpy as np
import matplotlib.pyplot as plt
import os.path
import shutil
from obspy import UTCDateTime
from netCDF4 import Dataset
from matplotlib import cm
from skimage.filters import gaussian
from mpl_toolkits.basemap import Basemap

dpi = 300
FigFormat = "png"

In [4]:
ExampleInputDir = '../Runs/modelball0000/input'

for imodel in np.arange(0,1):

    ModeName = 'modelballsimple%04d' %imodel
    EventParDir='../Runs/%s' %(ModeName)  
    if not os.path.exists(EventParDir):
        os.makedirs(EventParDir)
    print(EventParDir, " created")

    if not os.path.exists(EventParDir+'/input'):
        os.makedirs(EventParDir+'/input')

    # # generate random model

    # # Our 2-dimensional distribution will be over variables grid_lat and grid_lon
    scale = 2
    sigma = 1

    grid_radius = np.linspace(0, 1000, 5)
    grid_lat = np.linspace(-90, 90, 5)
    grid_lon = np.linspace(-180, 180, 9)

    # Make sure RADISU and Coordinates are ascendingly sorted
    grid_radius.sort()
    grid_lat.sort()
    grid_lon.sort()

    GridLAT, GridLON, GridDepth = np.meshgrid(grid_lat, grid_lon, grid_radius, indexing='ij')
    # MLtomo_ptb =np.random.normal(loc=0.0, scale=0.05, size=(len(grid_x), len(grid_y), len(depth)))

    # random uniform distribution between [0 - 1)
    tomorange = 0.1 
    MLtomo_ptb = (np.random.rand(len(grid_lat), len(grid_lon), len(grid_radius)) - 0.5)/0.5*tomorange
    ## smooth the random model 
    MLtomo_ptb_smooth = gaussian(MLtomo_ptb, sigma=sigma)
    # MLtomo_ptb[:,:,0] = np.zeros(np.shape(MLtomo_ptb[:,:,0]))
    print(MLtomo_ptb_smooth.min(), MLtomo_ptb_smooth.max())


    # write to file
    if os.path.exists(EventParDir+'/input/'+'ball_random_tomo.nc'):
        os.remove(EventParDir+'/input/'+'ball_random_tomo.nc')

    nc = Dataset(EventParDir+'/input/'+'ball_random_tomo.nc', 'w')
    nc.createDimension('nlat', size=len(grid_lat))
    nc.createDimension('nlon', size=len(grid_lon))
    nc.createDimension('nradius', size=len(grid_radius))
    nc.createVariable('latitude', float, dimensions=('nlat'))
    nc['latitude'][:] = grid_lat
    nc.createVariable('longitude', float, dimensions=('nlon'))
    nc['longitude'][:] = grid_lon
    nc.createVariable('radius', float, dimensions=('nradius'))
    nc['radius'][:] = grid_radius
    nc.createVariable('dvp', float, dimensions=('nlat', 'nlon','nradius'))
    nc['dvp'][:,:,:] = MLtomo_ptb_smooth[:,:,:]


    if imodel == 0:
        nc['dvp'][:,:,:] = np.zeros(np.shape(MLtomo_ptb_smooth[:,:,:]))
        nc.close()
        continue

    nc.close()

    # shutil.copy(ExampleInputDir+'/TomoNet_LowerMantle__10s.e',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/inparam.model.yaml',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/inparam.nr.yaml',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/inparam.advanced.yaml',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/inparam.source.yaml',EventParDir+'/input/')
    # # update event source
    # with open(EventParDir+'/input/inparam.source.yaml','r') as file:
    #     filetxt = file.read()
    # filetxt = filetxt.replace("latitude_longitude: [-56.24, 26.34]", "latitude_longitude: [%.2f, %.2f]" %(EventLat, EventLon))
    # filetxt = filetxt.replace("depth: 46.56e3", "depth: %.1f" %(EventDepth))
    # filetxt = filetxt.replace("data: [2.73e23, 9.08e21, -2.82e23, 6.30e22, 1.90e23, 2.16e23]", "data: [%e, %e, %e, %e, %e, %e]"  %(MomentTensor['m_rr'],MomentTensor['m_tt'],MomentTensor['m_pp'],MomentTensor['m_rt'],MomentTensor['m_rp'],MomentTensor['m_tp']))
    # with open(EventParDir+'/input/inparam.source.yaml','w') as file:
    #     file.write(filetxt)

    shutil.copy(ExampleInputDir+'/inparam.output.yaml',EventParDir+'/input/')

    # shutil.copy(ExampleInputDir+'/Synthetic_Stations',EventParDir+'/input/')



../Runs/modelballsimple0000  created
-0.02744470471116542 0.03522202171824493


In [7]:
MLtomo_ptb_smooth.min(), MLtomo_ptb_smooth.max()

(-0.04590417695446651, 0.04053691999487109)

In [8]:
MLtomo_ptb.min(), MLtomo_ptb.max()

(-0.09998533711018423, 0.09996325148616125)